In [2]:
# EDA 사전 세팅
%matplotlib inline

import time
import os
os.environ['KMP_WARNINGS'] = 'off'
from functools import reduce
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
import seaborn as sns  #데이터 시각화 
sns.set(font="AppleGothic", 

        rc={"axes.unicode_minus":False},
        
        style='darkgrid')

# 모델링
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import mean_absolute_error


from xgboost import plot_importance

In [1]:
train = pd.read_csv('/Users/choejeehyuk/분석에 필요한 데이터/신용커드 예측 경진대회/train.csv')
test = pd.read_csv('/Users/choejeehyuk/분석에 필요한 데이터/신용커드 예측 경진대회/test.csv')
train.head()

NameError: name 'pd' is not defined

In [179]:
train.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000000,26457.000000,2.645700e+04,26457.000000,26457.000000,26457.0,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000
mean,13228.000000,0.428658,1.873065e+05,-15958.053899,59068.750728,1.0,0.224742,0.294251,0.091280,2.196848,-26.123294,1.519560
std,7637.622372,0.747326,1.018784e+05,4201.589022,137475.427503,0.0,0.417420,0.455714,0.288013,0.916717,16.559550,0.702283
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6614.000000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19842.000000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,19.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,2.000000


In [38]:
train.isnull().sum()

index            0
gender           0
car              0
reality          0
child_num        0
income_total     0
income_type      0
edu_type         0
family_type      0
house_type       0
DAYS_BIRTH       0
DAYS_EMPLOYED    0
FLAG_MOBIL       0
work_phone       0
phone            0
email            0
occyp_type       0
family_size      0
begin_month      0
credit           0
dtype: int64

In [180]:

def make_derivation(data):
    data = data.fillna('NAN')
    data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else abs(x))

    feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
    for feat in feats:
        data[feat]=np.abs(data[feat])
    
    data = data[(data['family_size'] <= 8)]
    data = data.reset_index(drop=True)
    
    data['new_age'] = round(abs(data['DAYS_BIRTH'])/365.5,0).astype(np.int)
    data['year_os'] = data['DAYS_EMPLOYED']//365 # 근속연수
    data['month_os'] = data['DAYS_EMPLOYED']//30 # 근속월수
    data['month_ap'] = np.floor(data['DAYS_EMPLOYED']/30) - ((np.floor(data['DAYS_EMPLOYED']/30) / 12).astype('int') * 12) # 고용된 달 
    data['week_ap'] = np.floor(data['DAYS_EMPLOYED']/7) - ((np.floor(data['DAYS_EMPLOYED']/7) / 4).astype('int') * 4)
    data['aver_dep'] = data['income_total']/data['family_size'] 
    data['year_income'] = data['income_total'] // (data['year_os'])# 고용된 주
    data["family-child"] = data["family_size"] - data["child_num"]
    data.drop(['index'], axis=1, inplace = True)
    for i in range(len(data)):
        if data['family-child'][i] < 0:
            data['family-child'][i] == 0
        
    data = data.replace([np.inf],0)
        
    data['income_total']= data['income_total'].astype(int)
    data.family_size= data.family_size.astype(int)
    data.begin_month= data.begin_month.astype(int)
    data.drop(['FLAG_MOBIL','work_phone','phone', 'email'],axis=1,inplace=True)
    
    return data


In [181]:
train = make_derivation(train)
test = make_derivation(test)

In [155]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,occyp_type,family_size,begin_month,credit,new_age,year_os,month_os,month_ap,week_ap,aver_dep,year_income,family-child
0,F,N,N,0,202500,Commercial associate,Higher education,Married,Municipal apartment,13899,4709,NAN,2,6,1.0,38,12,156,0.0,0.0,101250.0,16875.0,2.0
1,F,N,Y,1,247500,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,Laborers,3,5,1.0,31,4,51,3.0,0.0,82500.0,61875.0,2.0
2,M,Y,Y,0,450000,Working,Higher education,Married,House / apartment,19087,4434,Managers,2,22,2.0,52,12,147,3.0,1.0,225000.0,37500.0,2.0
3,F,N,Y,0,202500,Commercial associate,Secondary / secondary special,Married,House / apartment,15088,2092,Sales staff,2,37,0.0,41,5,69,9.0,2.0,101250.0,40500.0,2.0
4,F,Y,Y,0,157500,State servant,Higher education,Married,House / apartment,15037,2105,Managers,2,26,2.0,41,5,70,10.0,0.0,78750.0,31500.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26446,F,N,N,2,225000,State servant,Secondary / secondary special,Married,House / apartment,12079,1984,Core staff,4,2,1.0,33,5,66,6.0,3.0,56250.0,45000.0,2.0
26447,F,N,Y,1,180000,Working,Higher education,Separated,House / apartment,15291,2475,NAN,2,47,2.0,42,6,82,10.0,1.0,90000.0,30000.0,1.0
26448,F,Y,N,0,292500,Working,Secondary / secondary special,Civil marriage,With parents,10082,2015,Core staff,2,25,2.0,28,5,67,7.0,3.0,146250.0,58500.0,2.0
26449,M,N,Y,0,171000,Working,Incomplete higher,Single / not married,House / apartment,10145,107,Laborers,1,59,2.0,28,0,3,3.0,3.0,171000.0,0.0,1.0


In [182]:
def list_sep (data):
    cat_list = []
    int_list = []
    for i in data.columns:
        if data[i].dtypes == 'object':
            cat_list.append(i)
        else :
            int_list.append(i)

    return int_list, cat_list

In [183]:
int_list, cat_list = list_sep(train)
print('int : ',int_list)
print('cat : ',cat_list)

int :  ['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'family_size', 'begin_month', 'credit', 'new_age', 'year_os', 'month_os', 'month_ap', 'week_ap', 'aver_dep', 'year_income', 'family-child']
cat :  ['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']


In [184]:
Encoder = LabelEncoder()
for i in cat_list:
    train[i] = Encoder.fit_transform(train[i].astype(str))
    test[i] = Encoder.transform(test[i].astype(str))

In [159]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,occyp_type,family_size,begin_month,credit,new_age,year_os,month_os,month_ap,week_ap,aver_dep,year_income,family-child
0,0,0,0,0,202500,0,1,1,2,13899,4709,12,2,6,1.0,38,12,156,0.0,0.0,101250.0,16875.0,2.0
1,0,0,1,1,247500,0,4,0,1,11380,1540,8,3,5,1.0,31,4,51,3.0,0.0,82500.0,61875.0,2.0
2,1,1,1,0,450000,4,1,1,1,19087,4434,10,2,22,2.0,52,12,147,3.0,1.0,225000.0,37500.0,2.0
3,0,0,1,0,202500,0,4,1,1,15088,2092,15,2,37,0.0,41,5,69,9.0,2.0,101250.0,40500.0,2.0
4,0,1,1,0,157500,2,1,1,1,15037,2105,10,2,26,2.0,41,5,70,10.0,0.0,78750.0,31500.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26446,0,0,0,2,225000,2,4,1,1,12079,1984,3,4,2,1.0,33,5,66,6.0,3.0,56250.0,45000.0,2.0
26447,0,0,1,1,180000,4,1,2,1,15291,2475,12,2,47,2.0,42,6,82,10.0,1.0,90000.0,30000.0,1.0
26448,0,1,0,0,292500,4,4,0,5,10082,2015,3,2,25,2.0,28,5,67,7.0,3.0,146250.0,58500.0,2.0
26449,1,0,1,0,171000,4,2,3,1,10145,107,8,1,59,2.0,28,0,3,3.0,3.0,171000.0,0.0,1.0


In [185]:
from sklearn.preprocessing import StandardScaler
tilist =['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'family_size', 'begin_month', 'new_age', 'year_os', 'month_os', 'month_ap', 'week_ap', 'aver_dep', 'year_income', 'family-child']
Scaler = StandardScaler()

train[tilist] = Scaler.fit_transform(train[tilist])
test[tilist] = Scaler.transform(test[tilist])

In [162]:
from imblearn.over_sampling import SMOTE
# r_train = train.copy()
target = train['credit'].values
smote = SMOTE(random_state=23)
train , target= smote.fit_resample(train, target)
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,occyp_type,family_size,begin_month,credit,new_age,year_os,month_os,month_ap,week_ap,aver_dep,year_income,family-child
0,0,0,0,-0.589850,0.149115,0,1,1,2,-0.490076,1.059065,12,-0.216262,-1.215242,1.0,-0.492903,0.994253,1.052910,-1.230046,-1.077087,0.002062,-0.414275,0.548245
1,0,0,1,0.795036,0.590785,0,4,0,1,-1.089581,-0.277867,8,0.897294,-1.275633,1.0,-1.101276,-0.250471,-0.277235,-0.424295,-1.077087,-0.254157,0.298249,0.548245
2,1,1,1,-0.589850,2.578304,4,1,1,1,0.744635,0.943049,10,-0.216262,-0.248981,2.0,0.723843,0.994253,0.938898,-0.424295,-0.223607,1.693108,-0.087702,0.548245
3,0,0,1,-0.589850,0.149115,0,4,1,1,-0.207101,-0.044990,15,-0.216262,0.656889,0.0,-0.232172,-0.094880,-0.049211,1.187206,0.629874,0.002062,-0.040200,0.548245
4,0,1,1,-0.589850,-0.292556,2,1,1,1,-0.219239,-0.039505,10,-0.216262,-0.007416,2.0,-0.232172,-0.094880,-0.036542,1.455790,-1.077087,-0.305401,-0.182705,0.548245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50881,0,0,1,1.649330,-0.207947,2,4,1,1,-0.607197,-0.529196,10,1.584213,-1.145828,1.0,-0.587144,-0.561651,-0.525743,0.752942,-0.223607,-0.735480,0.633693,0.548245
50882,1,1,1,-0.589850,1.694962,4,4,1,1,0.309107,0.437639,4,-0.216262,-1.085443,1.0,0.289291,0.371891,0.432175,1.724374,0.629874,1.078182,0.031052,0.548245
50883,0,1,1,-0.589850,0.590785,4,4,1,1,-0.009329,-0.039927,12,-0.216262,-1.419279,1.0,0.028559,-0.094880,-0.036542,1.455790,-1.077087,0.309525,0.102305,0.548245
50884,0,0,1,-0.589850,0.388727,4,4,2,1,-0.231345,-0.757236,12,-1.329817,-1.463281,1.0,-0.213697,-0.717242,-0.759160,-0.972881,-0.223607,1.719251,2.911440,-1.817824


In [163]:
print(len(train[train['credit'] == 2.0]))
print(len(train[train['credit'] == 1.0]))
print(len(train[train['credit'] == 0.0]))

16962
16962
16962


In [186]:
## feature_set 저장
feat_train = train.drop('credit', axis=1).values
target = train['credit'].values
feat_test = test.values
n_class = 3

In [187]:

X_train , X_valid , y_train , y_valid = train_test_split(feat_train , target , test_size=0.2)

In [188]:
xgb_pred_test = np.zeros((test.shape[0], 3))

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_valid)
y_pred_proba = model.predict_proba(X_valid)
confusion = confusion_matrix(y_valid, y_pred)
acc = accuracy_score(y_valid, y_pred)
logloss = log_loss(y_valid, y_pred_proba)

print(confusion, round(acc * 100, 2), logloss)

xgb_pred_test += model.predict_proba(feat_test)

[17:25:46] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[  59   92  523]
 [  23  421  839]
 [  32   70 3232]] 70.16 0.7608386787336494


In [189]:
n_fold = 10
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(feat_train.shape)

lgb_p_val = np.zeros((feat_train.shape[0], n_class))
lgb_p_tst = np.zeros((feat_test.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(feat_train, target), 1):
    print(f'training model for CV #{i}')
    lgb_clf = LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=230, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=2021,
                       verbose = 0)

    lgb_clf.fit(feat_train[i_trn], target[i_trn],
            eval_set=[(feat_train[i_val], target[i_val])])

    lgb_p_val[i_val, :] = lgb_clf.predict_proba(feat_train[i_val])
    lgb_p_tst += lgb_clf.predict_proba(feat_test) / n_fold
    
print(f'{log_loss(target, lgb_p_val)}')
print(f'{confusion_matrix(target, np.argmax(lgb_p_val, axis=1))}%')

(26451, 22)
training model for CV #1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003080 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's multi_logloss: 0.878144
[2]	valid_0's multi_logloss: 0.872717
[3]	valid_0's multi_logloss: 0.861673
[4]	valid_0's multi_logloss: 0.857301
[5]	valid_0's multi_logloss: 0.854045
[6]	valid_0's multi_logloss: 0.845138
[7]	valid_0's multi_logloss: 0.837127
[8]	valid_0's multi_logloss: 0.833559
[9]	valid_0's multi_logloss: 0.82739
[10]	valid_0's multi_logloss: 0.820052
[11]	valid_0's multi_logloss: 0.811042
[12]	valid_0's multi_logloss: 0.808777
[13]	valid_0's multi_logloss: 0.805682
[14]	valid_0's multi_logloss: 0.803433
[15]	valid_0's multi_logloss: 0.800548
[16]	valid_0's multi_logloss: 0.796049
[17]	valid_0's multi_logloss: 0.794438
[18]	valid_0's multi_logloss: 0.791969
[19]	valid_0's multi_logloss: 0.790005
[20

In [190]:
print(f'{log_loss(target, lgb_p_val)}')

0.6997121242948691


In [191]:
sub = pd.read_csv('/Users/choejeehyuk/분석에 필요한 데이터/신용커드 예측 경진대회/sample_submission.csv')

In [192]:
sub.iloc[:, 1:] = lgb_p_tst
sub

,index,0,1,2
0,26457,0.034973,0.083927,0.881100
1,26458,0.371576,0.298130,0.330294
2,26459,0.022536,0.054362,0.923102
3,26460,0.096943,0.096916,0.806141
4,26461,0.033081,0.081572,0.885347
...,...,...,...,...
9995,36452,0.082889,0.284993,0.632118
9996,36453,0.314017,0.214956,0.471027
9997,36454,0.013861,0.124895,0.861244
9998,36455,0.138487,0.296957,0.564555


In [193]:
sub.to_csv("1-29-17:09-lgbm_hyperop_nosmote_submission.csv", index=False)